# Supervised Fine-Tuning with PubMed Dataset on Microsoft Foundry

This notebook demonstrates how to fine-tune language models using **Supervised Fine-Tuning (SFT)** with the PubMed Medical Research Summarization dataset.

## What You'll Learn
1. Understand supervised fine-tuning for medical text summarization
2. Prepare and format medical research data
3. Upload datasets to Microsoft Foundry
4. Create and monitor a supervised fine-tuning job
5. Deploy and test your fine-tuned model

**Note**: Execute each cell in sequence.

## 1. Setup and Installation

Install all required packages from requirements.txt

In [1]:
pip install -r requirements.txt

  Using cached azure_ai_projects-2.0.0b2-py3-none-any.whl.metadata (63 kB)
  Using cached openai-2.14.0-py3-none-any.whl.metadata (29 kB)
  Using cached azure_identity-1.25.1-py3-none-any.whl.metadata (88 kB)
  Using cached azure_mgmt_cognitiveservices-14.1.0-py3-none-any.whl.metadata (32 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached isodate-0.7.2-py3-none-any.whl.metadata (11 kB)
  Using cached azure_core-1.37.0-py3-none-any.whl.metadata (47 kB)
  Using cached azure_storage_blob-12.27.1-py3-none-any.whl.metadata (26 kB)
  Using cached anyio-4.12.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.12.0-cp311-cp311-win_amd64.whl.metadata (5.3 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.w


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Import Libraries

In [1]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

print("All libraries imported successfully")

All libraries imported successfully


## 3. Configure Azure Environment

Set your Microsoft Foundry Project endpoint and model name. We're using **gpt-4.1** in this example, but you can use other supported GPT models. Copy the file `.env.template` (located in this folder), and save it as file named `.env`. Enter appropriate values for the environment variables used for the job you want to run.

```
MICROSOFT_FOUNDRY_PROJECT_ENDPOINT=<your-endpoint>
MODEL_NAME=gpt-4.1
AZURE_SUBSCRIPTION_ID=<your-subscription-id>
AZURE_RESOURCE_GROUP=<your-resource-group>
AZURE_AOAI_ACCOUNT=<your-foundry-account-name>
```

In [2]:
load_dotenv()

endpoint = os.environ.get("MICROSOFT_FOUNDRY_PROJECT_ENDPOINT")
model_name = os.environ.get("MODEL_NAME")

# Define dataset file paths
training_file_path = "training.jsonl"
validation_file_path = "validation.jsonl"

## 4. Connect to Microsoft Foundry Project

Connect to Microsoft Foundry Project using Azure credential authentication. This initializes the project client and OpenAI client needed for fine-tuning workflows.

**Important**: Ensure you have the **Azure AI User** role assigned to your account for the Microsoft Foundry Project resource.

In [3]:
credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=endpoint, credential=credential)
openai_client = project_client.get_openai_client()

print("Connected to Microsoft Foundry Project")

Connected to Microsoft Foundry Project


## 5. Upload Training Files

Upload the training and validation JSONL files to Microsoft Foundry. Each file is assigned a unique ID that will be referenced when creating the fine-tuning job.

In [5]:
print("Uploading training file...")
with open(training_file_path, "rb") as f:
    train_file = openai_client.files.create(file=f, purpose="fine-tune")
print(f"Training file ID: {train_file.id}")

print("\nUploading validation file...")
with open(validation_file_path, "rb") as f:
    validation_file = openai_client.files.create(file=f, purpose="fine-tune")
print(f"Validation file ID: {validation_file.id}")

Uploading training file...
Training file ID: file-05083606477b49b696a6500f066cd028

Uploading validation file...
Validation file ID: file-7aa581fd19ec4c959322612351f44638


## 6. Wait for File Processing

Microsoft Foundry needs to process the uploaded files before they can be used for fine-tuning.

In [6]:
print("Waiting for files to be processed...")
openai_client.files.wait_for_processing(train_file.id)
openai_client.files.wait_for_processing(validation_file.id)
print("Files ready!")

Waiting for files to be processed...
Files ready!


## 7. Create Supervised Fine-Tuning Job

Create a supervised fine-tuning job with your uploaded datasets. Configure the following hyperparameters to control the training process:

**Hyperparameters:**
1. **n_epochs (3)**: Number of complete passes through the training dataset. More epochs can improve performance but may lead to overfitting. Typical range: 1-10.
2. **batch_size (1)**: Number of training examples processed together in each iteration. Smaller batches provide more frequent updates. Typical range: 1-8.
3. **learning_rate_multiplier (1.0)**: Scales the default learning rate. Values < 1.0 make training more conservative, while values > 1.0 speed up learning but may cause instability. Typical range: 0.1-2.0.

**Note**: Adjust these based on your dataset size and quality.

In [7]:
print("Creating supervised fine-tuning job...")

fine_tune_job = openai_client.fine_tuning.jobs.create(
    model=model_name,
    training_file=train_file.id,
    validation_file=validation_file.id,    
    method={
        "type": "supervised",
        "supervised": {"hyperparameters": {"n_epochs": 3, "batch_size": 1, "learning_rate_multiplier": 1.0}},
    },
    extra_body={"trainingType": "GlobalStandard"},
    suffix="pubmed-summarization"
)

print(f"Fine-tuning job created!")
print(f"Job ID: {fine_tune_job.id}")
print(f"Status: {fine_tune_job.status}")
print(f"Model: {fine_tune_job.model}")

Creating supervised fine-tuning job...
Fine-tuning job created!
Job ID: ftjob-ccae5a26756f4dcb833f221c88e633c8
Status: pending
Model: gpt-4.1-2025-04-14


## 8. Monitor Training Progress

Track the status of your fine-tuning job. You can view the current status, and recent training events. Training duration varies based on dataset size, model, and hyperparameters - typically ranging from minutes to several hours.

In [7]:
job_status = openai_client.fine_tuning.jobs.retrieve(fine_tune_job.id)
print(f"Status: {job_status.status}")

Status: pending


## 9. Retrieve Fine-Tuned Model

After the fine-tuning job succeeded, retrieve the fine-tuned model ID. This ID is required to make inference calls with your customized model.

In [ ]:
completed_job = openai_client.fine_tuning.jobs.retrieve(fine_tune_job.id)

if completed_job.status == "succeeded":
    fine_tuned_model_id = completed_job.fine_tuned_model
    print(f"Fine-tuned Model ID: {fine_tuned_model_id}")
else:
    print(f"Status: {completed_job.status}")

## 10. Deploy the Fine-Tuned Model

Deploy the fine-tuned model to Azure OpenAI as a deployment endpoint. This step is required before making inference calls. The deployment uses GlobalStandard SKU with 50 capacity.

In [ ]:
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.cognitiveservices.models import Deployment, DeploymentProperties, DeploymentModel, Sku

subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")
resource_group = os.environ.get("AZURE_RESOURCE_GROUP")
account_name = os.environ.get("AZURE_AOAI_ACCOUNT")

deployment_name = "gpt-4o-pubmed-finetuned"

with CognitiveServicesManagementClient(credential=credential, subscription_id=subscription_id) as cogsvc_client:
    deployment_model = DeploymentModel(format="OpenAI", name=fine_tuned_model_id, version="1")
    deployment_properties = DeploymentProperties(model=deployment_model)
    deployment_sku = Sku(name="GlobalStandard", capacity=50)
    deployment_config = Deployment(properties=deployment_properties, sku=deployment_sku)
    
    print(f"Deploying fine-tuned model: {fine_tuned_model_id}")
    deployment = cogsvc_client.deployments.begin_create_or_update(
        resource_group_name=resource_group,
        account_name=account_name,
        deployment_name=deployment_name,
        deployment=deployment_config,
    )
    
    print("Waiting for deployment to complete...")
    deployment.result()

print(f"Model deployment completed: {deployment_name}")

## 11. Test Fine-Tuned Model

Test your fine-tuned model by generating a summary for a sample medical research article.

In [ ]:
test_article = """Background: Alzheimer's disease (AD) is a progressive neurodegenerative disorder characterized by cognitive decline and memory loss. Recent studies have suggested that neuroinflammation plays a crucial role in disease pathogenesis. Objective: This study investigated the role of microglial activation and inflammatory cytokines in early-stage Alzheimer's disease progression. Methods: We conducted a longitudinal study of 150 patients with mild cognitive impairment (MCI) over 36 months. Cerebrospinal fluid (CSF) samples were analyzed for inflammatory markers including IL-6, TNF-α, and IL-1β. Brain imaging using PET scans assessed microglial activation. Results: Patients who progressed to AD showed significantly elevated levels of IL-6 (p<0.001) and TNF-α (p<0.01) at baseline compared to stable MCI patients. Microglial activation was observed in the hippocampus and entorhinal cortex regions. Multivariate analysis revealed that combined inflammatory markers predicted AD conversion with 78% accuracy. Conclusion: Elevated neuroinflammatory markers and microglial activation in MCI patients are associated with increased risk of progression to Alzheimer's disease, suggesting potential therapeutic targets for early intervention."""

response = openai_client.responses.create(
    model=deployment_name,
    input=[
        {"role": "system", "content": "You are a medical research summarization assistant. Create concise, accurate abstracts of medical research articles that capture the key findings and clinical implications."},
        {"role": "user", "content": f"Summarize this medical research article:\n\n{test_article}"}
    ]
)

print(response.output_text)

##  Congratulations!

You've successfully fine-tuned a model for medical research summarization using the PubMed dataset!

### Next Steps:
1. **Test with more examples**: Try different medical research articles to evaluate performance
2. **Adjust hyperparameters**: Experiment with different epoch counts, batch sizes, or learning rates
3. **Deploy to production**: Integrate your fine-tuned model into healthcare applications
4. **Fine-tune further**: Use your own medical publications or clinical notes for specialized summarization
5. **Explore use cases**: Apply the model to systematic reviews, literature analysis, or clinical decision support